In [1]:

import sys,re 
import datetime
import pprint
from elasticsearch import Elasticsearch, helpers

from __future__ import division

%matplotlib inline

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

indices = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']

def time_filter(indices, last_days=1):
    if last_days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'jobs_archive_', '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

ind = time_filter(indices, 100)
print ("indices:", len(ind))
ind = ','.join(ind)
print(ind)

ValueError: unconverted data remains: _to_10

In [ ]:
s = {
    "size": 0,
    "query": {
        "filtered": {
            "filter": {
                "bool": {
                    "filter": [
                        { 
                            "match": {
                                "prodsourcelabel":"managed"
                            }
                        },
                        {
                            "match": {
                                "jobstatus": "finished"
                            }
                        },
                        {
                            "range": {
                                "nevents": {"gte": 1}
                            }
                        },
                        {
                            "range": {
                                "corecount": {"gte": 1}
                            }
                        },
                        {
                            "range": {
                                "wall_time": {"gt": 60}
                            }
                        }
                    ],
                    "must_not": [
                        {
                            "query": {
                                "wildcard": {
                                    "processingtype": "panda*"
                                }
                            }
                        },
                        {
                            "query": {
                                "wildcard": {
                                    "processingtype": "merge*"
                                }
                            }
                        },
                        {
                            "query": {
                                "wildcard": {
                                    "processingtype": "pmerge*"
                                }
                            }
                        }                 
                    ]
                }
            }
        }
    },
    "aggs": {
        "ptypes": {
            "terms": {
                "field": "processingtype",
                "size": 1000
            },
            "aggs": {
                "transf": {
                    "terms": {
                        "field": "transformation",
                        "size": 1000
                    },
                    "aggs": {
                        "atlrel": {
                            "terms": {
                                "field": "atlasrelease",
                                "size": 1000
                            },
                            "aggs": {
                                "site": {
                                    "terms": {
                                        "field": "computingsite",
                                        "size": 1000
                                    },
                                    "aggs": {
                                        "jtask": {
                                            "terms": {
                                                "field": "jeditaskid",
                                                "size": 100000
                                            },
                                            "aggs": {
                                                "1": {
                                                    "extended_stats": {
                                                        "script": "(doc['nevents'].value) ? doc['cpuconsumptiontime'].value/doc['nevents'].value : 0",
                                                        "lang": "expression"
                                                    }
                                                },
                                                "2": {
                                                    "extended_stats": {
                                                        "script": "(doc['nevents'].value) ? doc['wall_time'].value * doc['corecount'].value / doc['nevents'].value : 0",
                                                        "lang": "expression"
                                                    }
                                                },
                                                "3": {
                                                    "sum": {
                                                        "field": "cpuconsumptiontime"
                                                    }
                                                },
                                                "4": {
                                                    "sum": {
                                                        "script": "doc['wall_time'].value * doc['corecount'].value",
                                                        "lang": "expression"
                                                    }
                                                },
                                                "6": {
                                                    "avg": {
                                                        "field": "corecount"
                                                    }
                                                },
                                                "7": {
                                                    "sum": {
                                                        "field": "nevents"
                                                    }
                                                },
                                                "8": {
                                                    "avg": {
                                                        "script": "doc['cpuconsumptiontime'].value/doc['wall_time'].value/doc['corecount'].value",
                                                        "lang": "expression"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}
res = es.search(index=ind, body=s, request_timeout=12000)

In [2]:
print('took:',res['took'],'\tshards:', res['_shards'],'\thits:',res['hits'])
# print('size returned:',sys.getsizeof(res))

NameError: name 'res' is not defined